Note: this currently runs terribly slow. I'm attempting to speed up this with numpy but it's a WIP.


Data preparation: The 28x28 pixel images are flattened into a 784-dimensional vector. The pixel values are often normalized by dividing them by 255.
Model architecture: An MLP for MNIST typically includes:

    An input layer with 784 nodes, one for each pixel.
    One or more hidden layers with a specified number of neurons, often using an activation function like ReLU.
    An output layer with 10 nodes, representing the 10 possible digits (0-9), usually with a Softmax activation function to produce probabilities.

Training: The model is trained using a large dataset of labeled images. The process involves:

    Forward propagation: The input image vector is passed through the network to get an output prediction.
    Backpropagation: The difference between the prediction and the actual label is calculated, and this error is used to adjust the model's weights to improve future predictions.
    This process is repeated for multiple epochs, which are full passes through the entire dataset.

Evaluation: After training, the model's performance is evaluated on a separate, unseen test set to determine its accuracy.


In [1]:
import datasets as ds

import kamui as km
from kamui import nn
import random

# inspecting dataset before committing to download it
# ds_builder = ds.load_dataset_builder("ylecun/mnist")
# ds_builder.info.features

In [2]:
random.seed(1337)

In [3]:
# download dataset splits
train_ds = ds.load_dataset("ylecun/mnist", split="train")
test_ds = ds.load_dataset("ylecun/mnist", split="test")

In [4]:
# inspecting first image in test set
print(test_ds[0]["label"])
test_ds[0]["image"]

7


In [5]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 5}

In [6]:
# preprocess: convert png image to rgb
train_ds = train_ds.cast_column("image", ds.features.Image(mode="RGB"))
train_ds[0]

{'image': <PIL.Image.Image image mode=RGB size=28x28>, 'label': 5}

In [7]:
# preprocess: convert rgb image to grayscale pixel values and then kamui
def rgb_to_kamui(examples):
    out = []
    for image in examples["image"]:
        # convert to rgb optional but best practice
        rgb = list(image.convert("RGB").getdata())
        # 0-255 to 0.0-1.0
        rgb_norm = [(r / 255.0, g / 255.0, b / 255.0) for r, g, b in rgb]
        # convert to grayscale with luma formula: perceived brightness 0.0-1.0
        gray = [0.2989 * r + 0.5870 * g + 0.1140 * b for r, g, b in rgb_norm]
        km_value = [km.Value(g) for g in gray]
        out.append(km_value)
    examples["value"] = out
    return examples


train_ds = train_ds.with_transform(rgb_to_kamui)
train_ds[0]["value"]

[Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, gra

In [8]:
from collections.abc import Iterable


class MNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.MLP(28 * 28, [128, 64, 10])

    def __call__(self, x: Iterable[km.Value]) -> list[km.Value]:
        return self.mlp(x)

    def parameters(self) -> list[km.Value]:
        return self.mlp.parameters()

    def __repr__(self) -> str:
        return f"MNIST of 1 MLP [\n{self.mlp.__repr__()}\n]"


model = MNIST()
print(str(model))
print(len(model.parameters()))

MNIST of 1 MLP [
MLP of 3 layers [
Layer of 128 neurons [ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeur

In [ ]:
# train loop
def train(
    model: nn.Module,
    dataset: ds.Dataset,
    *,
    epochs: int,
    batch_size: int,
    lr: float,
):
    """
    Epoch means one full pass over the dataset. Batch size means how many examples
    to process before updating the model parameters. Loss is averaged over the entire
    dataset for reporting.
    """
    for epoch in range(epochs):
        total_loss = 0.0
        for i in range(0, len(dataset), batch_size):
            batch = dataset[i : i + batch_size]
            batch_loss = km.Value(0.0)
            for j in range(batch_size):
                x = batch["value"][j]
                y = batch["label"][j]
                pred = model(x)
                one_hot_y = nn.one_hot(km.Value(y), num_classes=10)
                sample_loss = nn.cross_entropy_loss(pred, one_hot_y)
                batch_loss += sample_loss
                print(f"        Sample {j + 1}/{batch_size}, Loss: {sample_loss.data}")
            avg_batch_loss = batch_loss / batch_size
            total_loss += avg_batch_loss.data
            # backward
            model.zero_grad()
            avg_batch_loss.backward()
            # update
            for p in model.parameters():
                p.data -= lr * p.grad
            print(
                f"    Batch {i // batch_size + 1}/{(len(dataset) + batch_size - 1) // batch_size}, Batch Avg Loss: {avg_batch_loss.data}"
            )
        avg_loss = total_loss / len(dataset)
        print(f"Epoch {epoch + 1}/{epochs}, Epoch Avg Loss: {avg_loss}")


train(model, train_ds, epochs=1, batch_size=32, lr=0.01)

        Sample 1/32, Loss: 146.76938028150693
        Sample 2/32, Loss: 74.05911415403693
        Sample 3/32, Loss: 128.2282904631777
        Sample 4/32, Loss: 37.951605874353845
        Sample 5/32, Loss: 161.70461968864632
        Sample 6/32, Loss: 162.50561603147554
        Sample 7/32, Loss: 123.37258486395741
        Sample 8/32, Loss: 187.55213310128883
        Sample 9/32, Loss: 74.73529308130591
        Sample 10/32, Loss: 29.67827878207492
        Sample 11/32, Loss: 175.2029602656365
        Sample 12/32, Loss: 145.6129192453495
        Sample 13/32, Loss: 186.5902570259361
        Sample 14/32, Loss: 168.73058804617614
        Sample 15/32, Loss: 47.781442780141674
        Sample 16/32, Loss: 62.1178013860235
        Sample 17/32, Loss: 117.43547039181674
        Sample 18/32, Loss: 55.87997002819936
        Sample 19/32, Loss: 116.09315968134312
        Sample 20/32, Loss: 135.26750354695827
        Sample 21/32, Loss: 89.83915880369356
        Sample 22/32, Loss: 50.84

In [ ]:
# preprocess test dataset
test_ds = test_ds.cast_column("image", ds.features.Image(mode="RGB"))
test_ds[0]

{'image': <PIL.Image.Image image mode=RGB size=28x28>,
 'label': 7,
 'value': [Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0.0),
  Value(data=0.0, grad=0

In [ ]:
test_ds = test_ds.with_transform(rgb_to_kamui)
test_ds[0]["value"]

[Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, grad=0.0),
 Value(data=0.0, gra

In [ ]:
def evaluate(model: nn.Module, dataset: ds.Dataset) -> float:
    correct = 0
    for i in range(len(dataset)):
        x = dataset[i]["value"]
        y = dataset[i]["label"]
        pred = model(x)
        pred_label = max(range(len(pred)), key=lambda k: pred[k].data)
        if pred_label == y:
            correct += 1
        print(f"Predicted: {pred_label}, Actual: {y}, Accuracy so far: {(correct / (i + 1) * 100):.3g}%")
    accuracy = correct / len(dataset)
    return accuracy


accuracy = evaluate(model, test_ds)
print(f"Test set accuracy: {accuracy * 100:.2f}%")

Predicted: 7, Actual: 7, Accuracy so far: 100%
Predicted: 9, Actual: 2, Accuracy so far: 50%
Predicted: 1, Actual: 1, Accuracy so far: 66.7%
Predicted: 6, Actual: 0, Accuracy so far: 50%
Predicted: 9, Actual: 4, Accuracy so far: 40%
Predicted: 1, Actual: 1, Accuracy so far: 50%
Predicted: 7, Actual: 4, Accuracy so far: 42.9%
Predicted: 6, Actual: 9, Accuracy so far: 37.5%
Predicted: 2, Actual: 5, Accuracy so far: 33.3%
Predicted: 2, Actual: 9, Accuracy so far: 30%
Predicted: 0, Actual: 0, Accuracy so far: 36.4%
Predicted: 1, Actual: 6, Accuracy so far: 33.3%
Predicted: 9, Actual: 9, Accuracy so far: 38.5%
Predicted: 6, Actual: 0, Accuracy so far: 35.7%
Predicted: 7, Actual: 1, Accuracy so far: 33.3%
Predicted: 1, Actual: 5, Accuracy so far: 31.2%
Predicted: 9, Actual: 9, Accuracy so far: 35.3%
Predicted: 3, Actual: 7, Accuracy so far: 33.3%
Predicted: 9, Actual: 3, Accuracy so far: 31.6%
Predicted: 4, Actual: 4, Accuracy so far: 35%
Predicted: 5, Actual: 9, Accuracy so far: 33.3%
Predi

KeyboardInterrupt: 